In [ ]:
import requests
import xml.etree.ElementTree as ET

def get_usd_rate_by_date(date_str):
    url = "https://api.cba.am/exchangerates.asmx"
    headers = {
        "Content-Type": "text/xml; charset=utf-8",
        "SOAPAction": "http://www.cba.am/ExchangeRatesByDateByISO"
    }

    # Construct the SOAP XML body
    body = f"""<?xml version="1.0" encoding="utf-8"?>
    <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
                   xmlns:xsd="http://www.w3.org/2001/XMLSchema"
                   xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
      <soap:Body>
        <ExchangeRatesByDateByISO xmlns="http://www.cba.am/">
          <date>{date_str}T00:00:00</date>
          <ISO>USD</ISO>
        </ExchangeRatesByDateByISO>
      </soap:Body>
    </soap:Envelope>"""

    # Send the SOAP request
    response = requests.post(url, headers=headers, data=body)

    if response.status_code != 200:
        print(f"HTTP error: {response.status_code}")
        print(response.text)
        return

    # Parse the XML response
    namespaces = {
        'soap': 'http://schemas.xmlsoap.org/soap/envelope/',
        'cba': 'http://www.cba.am/'
    }

    #print(namespaces)

    root = ET.fromstring(response.content)

    print(response.content)

    # Find the exchange rate info
    rate_elem = root.find('.//cba:ExchangeRate', namespaces)
    if rate_elem is None:
        print("USD exchange rate not found.")
        return
    
    #print(root)

    iso = rate_elem.find('cba:ISO', namespaces).text
    amount = rate_elem.find('cba:Amount', namespaces).text
    rate = rate_elem.find('cba:Rate', namespaces).text

    print(f"USD Exchange Rate on {date_str}:")
    print(f"  Rate    : {rate}")
    print(f"  Amount  : {amount}")


date_str = input("YYYY-MM-DD")

get_usd_rate_by_date(date_str)

b'<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><ExchangeRatesByDateByISOResponse xmlns="http://www.cba.am/"><ExchangeRatesByDateByISOResult><CurrentDate>2020-10-20T00:00:00</CurrentDate><NextAvailableDate>2020-10-21T00:00:00</NextAvailableDate><PreviousAvailableDate>2020-10-19T00:00:00</PreviousAvailableDate><Rates><ExchangeRate><ISO>USD</ISO><Amount>1</Amount><Rate>493.33</Rate><Difference>0.93</Difference></ExchangeRate></Rates></ExchangeRatesByDateByISOResult></ExchangeRatesByDateByISOResponse></soap:Body></soap:Envelope>'
USD Exchange Rate on 2020-10-20:
  Rate    : 493.33
  Amount  : 1
  Change  : 0.93
